In [ ]:
#最低限必要な情報:物件名・住所・間取り・家賃・階数
#取得予定
    #物件情報：物件名・住所・アクセス・築年数
    #部屋情報：階数・賃料/管理費・敷金/礼金・間取り/面積
#希望条件：都内で利便性の高い場所・予算20万円程度・2人暮らし2K以上
#検索条件：JR山手線沿線・予算25万円以下・2K以上の間取り


In [1]:
#すべての情報を取得する

#必要なライブラリのインポート
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
from tqdm import tqdm

#最後のページのページ数を取得する
root_url = 'https://airdoor.jp/list?li=d-91&ur=250000&fp=d-2_k-2_dk-2_ldk-3_k-3_dk-3_ldk&p={}'
res = requests.get(root_url.format(1))
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text, 'html.parser')
total_page = int(soup.find_all('li', class_='Search_paginatorItem__Wjqhg')[-1].text)
total_page

#物件を格納するリストを用意しておく
d_list = []

# 正常にHTMLが取得できれば以下のコードを実行
if res.status_code == 200:

    #変数urlにairdoorホームページのURLを1ページずつfor文で入れる
    for i in tqdm(range(1, total_page+1)):
        root_url = 'https://airdoor.jp/list?li=d-91&ur=250000&fp=d-2_k-2_dk-2_ldk-3_k-3_dk-3_ldk&p={}'
        res = requests.get(root_url.format(i))
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, 'html.parser')
        time.sleep(1)

        #ページ内の全物件の情報
        contents = soup.find('ul', class_='Search_panelList__HJLmE').find_all('li')

        #1ページ目の全物件の情報をfor文で取得する
        for content in contents:
            #物件名
            name = content.find('div', class_='PropertyPanelBuilding_buildingTitle__NbWmb')
            #住所
            info = content.find('div', 'PropertyPanelBuilding_buildingInformation__kRI1U is-sp-only')
            access1 = info
            address = info
            age = info
            height = info

            #物件の全部屋の情報をfor文で取得する
            rooms = content.find_all('div', class_='PropertyPanelRoom_roomItemUpside__BsYgr')
            for room in rooms:
                #家賃・管理費
                rent, admin = room.find('div', class_='PropertyPanelRoom_rentPrice__HO4Jp').text.replace('管理費','').replace(')','').split('(')
                #敷金・礼金
                deposit = room.find('div', class_='PropertyPanelRoom_initialPrices__naYEA').find_all('li')[0].text
                gratuity = room.find('div', class_='PropertyPanelRoom_initialPrices__naYEA').find_all('li')[1].text
                #何階の部屋
                try:
                    floor = str(int(int(room.find('span', class_='is-ml5').text.replace(' ', '').split('/')[0].replace('号室',''))/100))+'階'
                except Exception as e:
                    floor = room.find('span', class_='is-ml5').text.replace(' ', '').split('/')[0].replace('F','')+'階'
                #間取り・面積
                madori = room.find('span', class_='is-ml5').text.replace(' ', '').split('/')[1]
                menseki = room.find('span', class_='is-ml5').text.replace(' ', '').split('/')[2]

                #辞書dに部屋情報を1件ずつ格納する
                d = {
                    'name':name.text.split('】')[1],
                    'address':address.find_all('p')[1].text,
                    'access1':access1.find_all('p')[0].text,
                    'access2':'',
                    'access3':'',
                    'age':age.find_all('p')[2].text.replace(' ','').split('/')[0],
                    'height':height.find_all('p')[2].text.replace(' ','').split('/')[1],
                    'floor':floor,
                    'rent':rent,
                    'admin':admin,
                    'deposit':deposit,
                    'gratuity':gratuity.replace('無料','-'),
                    'madori':madori,
                    'menseki':menseki,
                }

                #部屋情報をd_listに1件ずつ格納
                d_list.append(d)
#DataFrameに物件情報を代入
df = pd.DataFrame(d_list)

100%|██████████| 5/5 [00:10<00:00,  2.18s/it]


In [3]:
# STEP2：dfをスプレッドシートに格納する

# 必要なライブラリのインポート
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe

# 認証のためのAPIキーを含むjsonファイル
google_spreadsheet_api = '自分のjsonファイルのパスを指定'
# 認証のためのアクセス先
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

# 認証情報をcredentialsに代入
credentials = Credentials.from_service_account_file(
    google_spreadsheet_api,
    scopes=scopes
)

# 認証情報を使ってGoogleSpreadsheetにアクセス
gs = gspread.authorize(credentials)

# 対象のスプレッドシートのファイルとシートを指定。この時アクセス権限注意。
SP_SHEET_KEY = '自分のスプレッドシートのキーを指定'
workbook = gs.open_by_key(SP_SHEET_KEY)
worksheet = workbook.worksheet('自分のシートを指定')

# 対象のシートにdfのデータを記入。但しindexは除く。
set_with_dataframe(worksheet, df, include_index=False)